In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
sys.path.append("..")

In [6]:
from encoding.parse_rooms import parse_rooms, parse_exits, dictify_rooms

In [8]:
rooms = parse_rooms("../encoding/dsl/rooms_for_alloy.txt")
exits = parse_exits("../encoding/dsl/exits_for_alloy.txt")
design = dictify_rooms(rooms, exits)

In [7]:
from rom_tools.rom_manager import RomManager
from abstraction_validation.sm_paths import *

In [5]:
rom = RomManager("../../roms/sm_clean.sfc", "sm_junk.smc")
parsed_rom = rom.parse()

In [9]:
all_posns = all_positions(rooms, parsed_rom, is_global=False)

Landing_Site 0x791f8
	 Extra: []
	 Missing: ['Landing_Site_Ship']
West_Ocean 0x793fe
	 Extra: ['West_Ocean_R4', 'West_Ocean_L']
	 Missing: ['West_Ocean_R6', 'West_Ocean_L2']
Bomb_Torizo 0x79804
	 Extra: ['Bomb_Torizo_B']
	 Missing: ['Bomb_Torizo_Bombs']
Spore_Spawn 0x79dc7
	 Extra: []
	 Missing: ['Spore_Spawn_Spore_Spawn']
Kraid 0x7a59f
	 Extra: []
	 Missing: ['Kraid_Kraid']
Statues 0x7a66a
	 Extra: []
	 Missing: ['Statues_Statues']
Crocomire 0x7a98d
	 Extra: []
	 Missing: ['Crocomire_Crocomire']
Water_Closet 0x7b1e5
	 Extra: []
	 Missing: ['Water_Closet_Drain']
Golden_Torizo 0x7b283
	 Extra: []
	 Missing: ['Golden_Torizo_Golden_Torizo']
Ridley 0x7b32e
	 Extra: []
	 Missing: ['Ridley_Ridley']
Phantoon 0x7cd13
	 Extra: []
	 Missing: ['Phantoon_Phantoon']
West_Sand_Hall 0x7d461
	 Extra: []
	 Missing: ['West_Sand_Hall_TS']
East_Sand_Hall 0x7d4c2
	 Extra: []
	 Missing: ['East_Sand_Hall_TS']
West_Sand_Hole 0x7d4ef
	 Extra: []
	 Missing: ['West_Sand_Hole_TS']
East_Sand_Hole 0x7d51e
	 Extra: 

In [10]:
#rooms: [room_name]
#room.png
#room.json:
#{   name: "aa",
    #region: n,
    #mem_address: n,
    #map_pos: (x,y) # Local
    #map_size: (x,y)
#}

In [11]:
import json

In [12]:
json.dumps("(1,2)")

'"(1,2)"'

In [77]:
#TODO: items.json?
def mk_room_json(room, room_header):
    d = {
        "Name": room.name,
        "Region": room_header.area_index,
        "Memory_Address": room.mem_address,
        "Map_Pos": (room_header.map_x, room_header.map_y),
        "Extent": (room_header.width, room_header.height),
    }
    return d

In [34]:
from rom_tools import graphics
from PIL import Image

In [35]:
tileset_table = graphics.get_tileset_table(rom)
cre_tile_sheet = graphics.get_cre_tilesheet(rom)
cre_tile_table = graphics.get_cre_tile_table(rom)
tile_cache = {}

def get_tiles(tileset):
    if tileset in tile_cache:
        sce_tiles, cre_tiles = tile_cache[tileset]
    else:
        uc_tile_table, uc_tile_sheet, uc_palette = graphics.get_tileset(rom, tileset, tileset_table)
        sce_tiles = graphics.tile_images(uc_tile_table, uc_tile_sheet, cre_tile_sheet, uc_palette)
        cre_tiles = graphics.tile_images(cre_tile_table, uc_tile_sheet, cre_tile_sheet, uc_palette)
        tile_cache[tileset] = (sce_tiles, cre_tiles)
    return sce_tiles, cre_tiles

In [51]:
def get_level_image(header):
    d_state = header.state_chooser.default
    tileset = d_state.tileset
    level = d_state.level_data.level_array
    layer1 = level.layer1
    sce_tiles, cre_tiles = get_tiles(tileset)
    # TODO: add this to graphics.py
    l_image = Image.new("RGBA", (layer1.shape[0] * 16, layer1.shape[1] * 16), "black")
    xdim, ydim = layer1.shape
    for y in range(ydim):
        for x in range(xdim):
            tile = layer1[x][y]
            tile_image = graphics.get_tile(tile.texture.texture_index, sce_tiles, cre_tiles)
            #TODO: flips are parsed backwards?
            flips = (tile.texture.hflip, tile.texture.vflip)
            tile_image = graphics.apply_flips(tile_image, flips)
            # Third argument is the alpha mask
            l_image.paste(tile_image, (x * 16, y * 16), tile_image)
    return l_image

In [21]:
room_headers = [v for k,v in parsed_rom.items() if k.startswith("room_header")]

In [82]:
def write_room_info(rooms):
    all_rooms = {}
    for room in rooms:
        print(room.name)
        header = parsed_rom[f"room_header_{hex(room.mem_address)}"]
        room_json = mk_room_json(room, header)
        #room_image = get_level_image(header)
        with open(f"../output/room_info/{room.name}.json", "w") as f:
            json.dump(room_json, f)
        all_rooms[room.name] = room_json
        #room_image.save(f"../output/room_info/{room.name}.png")
    with open("../output/room_info/all_rooms.json", "w") as f:
        json.dump(all_rooms, f)

In [55]:
r = rooms["Landing_Site"]

In [65]:
#TODO: what's up with Kraid??
exceptions = ["Kraid"]
rtp = [r for k,r in rooms.items() if k not in exceptions]

In [83]:
write_room_info(rtp)

Landing_Site
Gauntlet_Entrance
Parlor
Crateria_Power_Bombs
Parlor_Save_Room
West_Ocean
Bowling_Path
Crateria_Kihunters
Crateria_Maridia_Shaft
East_Ocean
Forgotten_Highway
Crab_Maze
Crateria_Power_Door
Crateria_Tube
Moat
Caterpillar_Elevator
Gauntlet
Ocean_Ship_Trans
Climb_Room
Old_Mother_Brain
Blue_Brinstar_Elevator
Bomb_Torizo
BT_Flyway
Map_Flyway
Terminator_Room
Brinstar_Elevator
Kago_Room
Crateria_Map
Green_Pirates_Shaft
Crateria_Supers
Final_Missile_Bombway
Final_Missile
Brinstar_Entrance
Spore_Spawn_Supers
Brinstar_Map_Entrance
Early_Supers
Brinstar_Reserve
Brinstar_Map
Brinstar_Firefleas
Brinstar_Missile_Refill
Dachora_Room
Big_Pink
PreSpore_Kihunters
Spore_Spawn
Pink_Power_Bombs
Green_Hill_Zone
Morph_Ball_Room
Construction_Zone
Blue_Brinstar_Energy
Noob_Bridge
Green_Brinstar_Beetoms
Green_Brinstar_Energy
Etecoon_Supers
Brinstar_Refill
Spore_Spawn_Farm
Waterway_Energy
First_Missile
Pink_Hoppers
Hopper_Energy
Pink_Save
Blue_Brinstar_Boulders
Billy_Mays
Green_Entrance_Save
Green_Be

In [67]:
from abstraction_validation.abstractify import area_offsets

In [68]:
area_offsets

{0: c(3,10),
 1: c(0,29),
 2: c(31,49),
 3: c(37,0),
 4: c(28,29),
 5: c(0,10),
 6: c(0,-10),
 7: c(0,0)}

In [72]:
lxs = []
lys = []
rxs = []
rys = []
for room in rooms.values():
    header = parsed_rom[f"room_header_{hex(room.mem_address)}"]
    offx, offy = area_offsets[header.area_index]
    lxs.append(offx + header.map_x)
    lys.append(offy + header.map_y)
    rxs.append(offx + header.map_x + header.width)
    rys.append(offy + header.map_y + header.height)

In [73]:
print(min(lxs), max(rxs))
print(min(lys), max(lys))

5 71
10 66


In [76]:
# 16 pixels per tile * 16 tiles per maptile * 72 maptiles = 
f"{16 * 16 * 72} pixels wide"
f"{16 * 16 * 66} pixels wide"

'16896 pixels wide'

In [ ]:
# +y ^  +y v
# +x >  +x >
maxy - y

In [ ]:
# const ls_info = await fetch("/room_info/Landing_Site").then(res => res.json())

In [ ]:
#TODO: reupload images with alpha
#TODO: how to handle overlapping rooms?

In [97]:
9216 - 8960

256

In [96]:
140.8 - 138.24

2.5600000000000023